In [1]:
import tensorflow as tf
import numpy as np

## 1.数据预处理：

首先用numpy.load将数据读入，通过对数据进行分析，发现训练数据一共是334条，特征向量为10000维的数据。其中119条数据为label=0的数据，即健康猪肉，另外215条数据为label=1的数据，即病死猪肉。

然后进行数据预处理，采用numpy.vstack和numpy.hstack将数据预处理为tensorflow框架的输入格式。

结果如下：


In [2]:
# 训练数据
data = np.load("train0.npy")  # m=119, n=10000
data1 = np.load("train1.npy") # m=215, n=10000
print(data.shape, data1.shape)
X = np.vstack([data,data1]) # np.vstack() 在垂直方向上合并 119+215
print(X.shape)

(119, 10000) (215, 10000)
(334, 10000)


In [3]:
y1 = np.zeros((119,1))   # 对应train0  标签为0
y1temp = np.ones((119,1))
y1 = np.hstack((y1temp,y1)) # 水平方向上合并 119,1 + 119,1 = 119,2
y2 = np.ones((215,1))    # 对应train1  标签为1
y2temp = np.zeros((215,1))
y2 = np.hstack((y2temp,y2))
Y = np.vstack((y1,y2))
print(Y.shape)

(334, 2)


In [4]:
Y[0]

array([1., 0.])

预处理后标签Y 

其中，标签为[1,0]的向量标记为label=0，即健康的猪肉，标签为[0，1]的向量标记为label=1，即病死的猪肉。

In [5]:
# 测试数据
testdata0 = np.load("test0a.npy")  # m1= 20,n1= 10000
testdata1 = np.load("test1a.npy") # m1= 20,n1= 10000
xt = np.vstack((testdata0,testdata1))
xt.shape

(40, 10000)

In [6]:
yt1 = np.zeros((20,1))
yt1temp = np.ones((20,1))
yt1 = np.hstack((yt1temp,yt1))
yt2 = np.ones((20,1))
yt2temp = np.zeros((20,1))
yt2 = np.hstack((yt2temp,yt2))
yt = np.vstack((yt1,yt2))
yt.shape

(40, 2)

## 神经网络

采用Tensorflow搭建神经网络框架：通过tf.Variable设置神经网络的连接权值weights和偏置biases。

通过tf.reduce_sum函数来指定损失函数为预测值和真实值的交叉熵，

然后通过梯度下降优化器tf.GradientDescent来最小化交叉熵。

确定学习率为0.001，训练轮次为4000轮，每轮训练8条数据。


In [7]:
# 设置权重weights和偏置biases作为优化变量，初始值设为0
weights = tf.Variable(tf.zeros([10000,2]))
biases = tf.Variable(tf.zeros([2]))

In [8]:
# 构建神经网络模型
x = tf.placeholder('float', [None, 10000])
y = tf.placeholder('float', [None, 2])
z = tf.matmul(x, weights) + biases
pred = tf.nn.softmax(z)

In [9]:
# 预测值与真实值的交叉熵
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = pred))

# 使用梯度下降优化器最小化交叉熵
train = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
# train =  tf.train.AdamOptimizer(0.005).minimize(loss)

In [10]:
# 开始训练
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(4001):
    sess.run(train, feed_dict={x:X, y:Y})
    if i % 500 == 0:
        predition = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
        acc = tf.reduce_mean(tf.cast(predition, 'float'))
        print('准确率：', sess.run(acc, feed_dict={x:xt, y:yt}))

准确率： 0.5
准确率： 0.775
准确率： 0.775
准确率： 0.8
准确率： 0.8
准确率： 0.8
准确率： 0.8
准确率： 0.775
准确率： 0.775


修改这段带代码，使用训练集进行验证

print(sess.run(accuracy, feed_dict={x:X, y_real:Y}))

In [11]:
# 设置权重weights和偏置biases作为优化变量，初始值设为0
# 设置权重weights和偏置biases作为优化变量，初始值设为0
weights = tf.Variable(tf.zeros([10000,2]))
biases = tf.Variable(tf.zeros([2]))

# 构建神经网络模型
x = tf.placeholder('float', [None, 10000])
y = tf.placeholder('float', [None, 2])
z = tf.matmul(x, weights) + biases
pred = tf.nn.softmax(z)

# 预测值与真实值的交叉熵
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = pred))
loss = -tf.reduce_mean(y * tf.log(pred))

# 使用梯度下降优化器最小化交叉熵
# train =  tf.train.AdamOptimizer(0.01).minimize(loss)
train = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

# 开始训练
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
batch_size = 8
for i in range(10001):
    start = (i*batch_size) % 334
    end = start + batch_size
    sess.run(train, feed_dict={x:X[start:end], y:Y[start:end]})
    if i % 200 == 0:
        predition = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
        acc = tf.reduce_mean(tf.cast(predition, 'float'))
        print('准确率：', sess.run(acc, feed_dict={x:xt, y:yt}))

准确率： 0.5
准确率： 0.5
准确率： 0.775
准确率： 0.875
准确率： 0.825
准确率： 0.725
准确率： 0.8
准确率： 0.875
准确率： 0.875
准确率： 0.825
准确率： 0.8
准确率： 0.875
准确率： 0.875
准确率： 0.875
准确率： 0.85
准确率： 0.825
准确率： 0.875
准确率： 0.875
准确率： 0.875
准确率： 0.825
准确率： 0.85
准确率： 0.875
准确率： 0.875
准确率： 0.875
准确率： 0.825
准确率： 0.825
准确率： 0.875
准确率： 0.875
准确率： 0.85
准确率： 0.825
准确率： 0.825
准确率： 0.875
准确率： 0.875
准确率： 0.85
准确率： 0.825
准确率： 0.825
准确率： 0.875
准确率： 0.85
准确率： 0.825
准确率： 0.825
准确率： 0.825
准确率： 0.85
准确率： 0.85
准确率： 0.825
准确率： 0.825
准确率： 0.825
准确率： 0.85
准确率： 0.825
准确率： 0.825
准确率： 0.825
准确率： 0.825


实验结果：通过Tensorflow搭建的神经网络，设置每100轮输出结果，最终训练数据的准确率达到98.50%，测试数据的准确率达到85%。


## 实验总结：

通过上述实验，可以发现对于输入数据的特征向量为10000维的数据，神经网络仍可达到较好的拟合效果。对于测试数据为的准确率仅为85%，而训练数据的准确率高达98%，说明该网络对训练数据的学习过于深入，局部点陷入过拟合，导致鲁棒性较差。

后期改进可以通过对损失函数加入惩罚项，即正则化的方式，还有采用随机梯度下降方式，并且对神经网络的连接参数进行一定的修改，可以一定程度上解决这个问题。


## 正则化

正则化是在神经网络计算损失值的过程中，在损失后面再加上一项。这样损失值所代表的输出与标准结果间的误差就会受到干扰，导致学习参数w 和 b无法按照目标方向来调整，实现模型无法与样本完全拟合，从而达到防止过拟合的效果。正则化主要有L1和L2正则，如下：

    L1:所有学习参数w的绝对值的和

    L2:所有学习参数w的平方和然后求平方根。

在TensorFlow中，已经封装好了相应的函数，

    L2的正则化函数为：tf.nn.l2_loss(t, name=None)，
    L1的正则化函数需要自己组合，tf.reduce_sum(tf.abs(w))

In [13]:
# 设置权重weights和偏置biases作为优化变量，初始值设为0
# 设置权重weights和偏置biases作为优化变量，初始值设为0
weights = tf.Variable(tf.zeros([10000,2]))
biases = tf.Variable(tf.zeros([2]))

# 构建神经网络模型
x = tf.placeholder('float', [None, 10000])
y = tf.placeholder('float', [None, 2])
z = tf.matmul(x, weights) + biases
pred = tf.nn.softmax(z)

# 预测值与真实值的交叉熵
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = pred))
# loss = -tf.reduce_mean(y * tf.log(pred))
L2 = tf.nn.l2_loss(weights)
loss = -tf.reduce_mean(y * tf.log(pred)) + L2 * 0.01 # 正则化

# 使用梯度下降优化器最小化交叉熵
# train =  tf.train.AdamOptimizer(0.01).minimize(loss)
train = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

# 开始训练
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
batch_size = 8
for i in range(10001):
    start = (i*batch_size) % 334
    end = start + batch_size
    sess.run(train, feed_dict={x:X[start:end], y:Y[start:end]})
    if i % 200 == 0:
        predition = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
        acc = tf.reduce_mean(tf.cast(predition, 'float'))
        print('准确率：', sess.run(acc, feed_dict={x:xt, y:yt}))

准确率： 0.5
准确率： 0.5
准确率： 0.75
准确率： 0.875
准确率： 0.8
准确率： 0.725
准确率： 0.8
准确率： 0.875
准确率： 0.875
准确率： 0.825
准确率： 0.75
准确率： 0.825
准确率： 0.875
准确率： 0.875
准确率： 0.825
准确率： 0.825
准确率： 0.85
准确率： 0.875
准确率： 0.875
准确率： 0.825
准确率： 0.825
准确率： 0.85
准确率： 0.875
准确率： 0.875
准确率： 0.825
准确率： 0.825
准确率： 0.875
准确率： 0.875
准确率： 0.85
准确率： 0.825
准确率： 0.825
准确率： 0.875
准确率： 0.875
准确率： 0.825
准确率： 0.825
准确率： 0.825
准确率： 0.875
准确率： 0.875
准确率： 0.825
准确率： 0.825
准确率： 0.825
准确率： 0.875
准确率： 0.85
准确率： 0.825
准确率： 0.825
准确率： 0.825
准确率： 0.875
准确率： 0.825
准确率： 0.825
准确率： 0.825
准确率： 0.825


在使用正则化的时候，我们为正则化项设置一个权重的系数，注意这个权重系数的值，可以通过不断尝试来确定权重系数的值。